In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pathlib import Path

load_dotenv(override=True)

OPENAI_API_KEY = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]
MONGO_URI = os.environ["MONGO_URI"]
EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]
EMBEDDING_DIMENSIONS = os.environ["EMBEDDING_DIMENSIONS"]
CHAT_MODEL_NAME = os.environ["CHAT_MODEL_NAME"]

DB_NAME = "gaia"
COLLECTION_NAME = "documents"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
MAX_CHUNKS_TO_RETRIEVE=10
CHUNK_MIN_RELEVANCE_SCORE=0.2

MAX_TOKENS_FOR_RESPONSE = 500
CHAT_MODEL_TEMPERATURE=0.3
CHAT_MODEL_FREQ_PENALTY=0
CHAT_MODEL_PRES_PENALTY=0
MEMORY_WINDOW_SIZE = 5  # Number of turns to keep in memory before summarizing
SHOW_VERBOSE=True


PARENT_PATH = Path.cwd().parent
PROJECT_SPECIFIC_TEMPLATE_FOLDER = PARENT_PATH / 'static'

# History storage for testing in Jupyter
conversation_history = ""

In [11]:
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.runnables import RunnableSequence
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory

class RAG:
    def __init__(self, chat_data):
        self.chat_data = chat_data

    ## Public Methods
    def get_response(self):
        intent_name = self._detect_intent()
        qa = self._get_qa_instance(intent_name)
        result = qa.invoke({"question": self.chat_data["user_input"]})  # Fixed: Accessing dict key

        response_text = result.get("answer", "")
        sources_list = result.get("sources", "")

        return response_text, sources_list

    ## Private Methods
    def _load_template(self, folder, filename):
        file_path = os.path.join(folder, filename)
        with open(file_path, "r") as file:
            return file.read()

    def _build_intent_detection_prompt(self):
        # Load intent detection template
        intent_detection_template = self._load_template(
            self.chat_data["prompt_template_directory_name"],  # Fixed: Accessing dict key
            self.chat_data["intent_detection_prompt_template_file_name"]  # Fixed: Accessing dict key
        )

        # Dynamically build the intent list from intent details
        intent_list = "\n".join(
            [f'- "{intent_name}": {intent_data["description"]}' for intent_name, intent_data in self.chat_data["intent_details"].items()]  # Fixed: Accessing dict keys
        )

        # Build the full prompt for intent detection
        prompt = intent_detection_template.format(
            user_input=self.chat_data["user_input"],  # Fixed: Accessing dict key
            history=self.chat_data["chat_history"],  # Fixed: Accessing dict key
            intent_list=intent_list
        )
        return prompt

    def _build_chat_prompt(self, intent_name):
        # Load base template
        base_template = self._load_template(self.chat_data["prompt_template_directory_name"], self.chat_data["base_prompt_template_file_name"])  # Fixed: Accessing dict keys

        # Load intent-specific template or default to generic message
        if intent_name == "none":
            intent_template = "No specific intent. This is a generic message or continuation of the conversation."
        else:
            intent_template = self._load_template(self.chat_data["prompt_template_directory_name"], f"{intent_name}.txt")  # Fixed: Accessing dict key

        # Build the full prompt using the base and intent templates
        return base_template.format(
            subinstructions=intent_template,
            history=self.chat_data["chat_history"],  # Fixed: Accessing dict key
            summaries="{summaries}",
            question="{question}"
        )

    def _get_qa_retriever(self):
        # Initialize embeddings with settings from LLMSettings
        llm_embeddings = OpenAIEmbeddings(
            model=self.chat_data["llm_settings"]["embedding_model_name"],  # Fixed: Accessing dict key
            openai_api_key=self.chat_data["llm_settings"]["llm_key"]  # Fixed: Accessing dict key
        )

        vector_store= MongoDBAtlasVectorSearch.from_connection_string(
            self.chat_data["db_settings"]["uri"],
            self.chat_data["db_settings"]["db_name"] + "." + self.chat_data["db_settings"]["collection_name"],
            embedding=llm_embeddings,
            index_name=self.chat_data["db_settings"]["vector_index_name"]
        )

        # Configure retriever with settings from RAGSettings
        qa_retriever = vector_store.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                "k": self.chat_data["rag_settings"]["max_chunks_to_retrieve"],  # Fixed: Accessing dict key
                "score_threshold": self.chat_data["rag_settings"]["retrieved_chunks_min_relevance_score"] # Fixed: Accessing dict key
            }
        )
        return qa_retriever

    def _get_qa_instance(self, intent_name):
        # Build chat prompt for the detected intent
        dynamic_prompt_content = self._build_chat_prompt(intent_name)

        # Create the prompt template
        prompt_template = PromptTemplate(
            template=dynamic_prompt_content,
            input_variables=['summaries', 'question']
        )

        # Initialize the chat model using settings from RAGSettings
        llm_eva = ChatOpenAI(
            model_name=self.chat_data["rag_settings"]["chat_model_name"],  # Fixed: Accessing dict key
            temperature=self.chat_data["rag_settings"]["temperature"],  # Fixed: Accessing dict key
            max_tokens=self.chat_data["rag_settings"]["max_tokens_for_response"],  # Fixed: Accessing dict key
            openai_api_key=self.chat_data["llm_settings"]["llm_key"],  # Fixed: Accessing dict key
            streaming=False
        )

        # Get retriever and memory for the conversation
        qa_retriever = self._get_qa_retriever()
        memory = ConversationSummaryBufferMemory(
            memory_key="history",
            input_key="question",
            llm=llm_eva
        )

        # Build the RAG chain
        qa = RetrievalQAWithSourcesChain.from_chain_type(
            llm=llm_eva,
            chain_type="stuff",
            retriever=qa_retriever,
            return_source_documents=False,
            chain_type_kwargs={
                "verbose": SHOW_VERBOSE,
                "prompt": prompt_template,
                "memory": memory
            }
        )
        return qa

    def _detect_intent(self):
        # Build the intent detection prompt
        intent_prompt = self._build_intent_detection_prompt()

        # Initialize the LLM model for intent detection
        llm_eva = ChatOpenAI(
            model_name=self.chat_data["rag_settings"]["chat_model_name"],  # Fixed: Accessing dict key
            temperature=self.chat_data["rag_settings"]["temperature"],  # Fixed: Accessing dict key
            max_tokens=self.chat_data["rag_settings"]["max_tokens_for_response"],  # Fixed: Accessing dict key
            openai_api_key=self.chat_data["llm_settings"]["llm_key"],  # Fixed: Accessing dict key
            streaming=False
        )

        # Create the intent detection prompt template
        prompt_template = PromptTemplate(
            template=intent_prompt,
            input_variables=["user_input", "history", "intent_list"]
        )

        # Detect the intent by invoking the chain
        intent_chain = RunnableSequence(prompt_template, llm_eva)
        intent_result = intent_chain.invoke({
            "user_input": self.chat_data["user_input"],  # Fixed: Accessing dict key
            "history": self.chat_data["chat_history"],  # Fixed: Accessing dict key
            "intent_list": "\n".join([f'- "{intent_name}"' for intent_name in self.chat_data["intent_details"].keys()])  # Fixed: Accessing dict key
        })

        # Access content from AIMessage and handle potential "none" category
        detected_intent = intent_result.content.strip()
        if detected_intent not in self.chat_data["intent_details"]:  # Fixed: Accessing dict key
            return "none"
        return detected_intent


In [12]:
def handle_user_input(user_input_text):
    global conversation_history

    data = {
        "db_type": "mongodb",
        "db_settings": {
            "uri": MONGO_URI,  
            "db_name": DB_NAME,  
            "collection_name": COLLECTION_NAME,  
            "vector_index_name": ATLAS_VECTOR_SEARCH_INDEX_NAME,  
            "vector_similarity_function": "cosine"  
        },
        "llm_settings": {
            "llm_key": OPENAI_API_KEY,  
            "vector_dimension_size": EMBEDDING_DIMENSIONS,  
            "embedding_model_name": EMBEDDING_MODEL_NAME  
        },
        "rag_settings": {
            "chat_model_name": CHAT_MODEL_NAME,  
            "max_chunks_to_retrieve": MAX_CHUNKS_TO_RETRIEVE,  
            "retrieved_chunks_min_relevance_score": CHUNK_MIN_RELEVANCE_SCORE,
            "max_tokens_for_response": MAX_TOKENS_FOR_RESPONSE,  
            "temperature": CHAT_MODEL_TEMPERATURE,  
            "frequency_penalty": CHAT_MODEL_FREQ_PENALTY,
            "presence_penalty": CHAT_MODEL_PRES_PENALTY
        },
        "user_input": user_input_text,
        "chat_history": conversation_history,
        "prompt_template_directory_name": str(PROJECT_SPECIFIC_TEMPLATE_FOLDER),  
        "base_prompt_template_file_name": "base_template.txt", 
        "intent_detection_prompt_template_file_name": "detect_intent.txt", 
        "intent_details": {
            "diagnosis": {
                "filename": "diagnosis.txt",
                "description": "This intent covers diagnosis-related queries for crop issues."
            },
            "symptoms_identification": {
                "filename": "symptoms_identification.txt",
                "description": "This intent helps identify symptoms for a specific pest or problem."
            },
            "pest_list": {
                "filename": "pest_list.txt",
                "description": "This intent provides a list of pests affecting specific crops in a given location."
            },
            "ipm_pest_management": {
                "filename": "ipm_pest_management.txt",
                "description": "This intent provides integrated pest management advice, including biocontrol and chemical recommendations."
            },
            "chemical_handling_safety": {
                "filename": "chemical_handling_safety.txt",
                "description": "This intent provides safety advice regarding the handling and application of chemicals."
            },
            "invasive_pest_status": {
                "filename": "invasive_pest_status.txt",
                "description": "This intent provides the current status of invasive pests in a specific region."
            },
            "dosage_recommendations": {
                "filename": "dosage_recommendations.txt",
                "description": "This intent provides dosage recommendations for specific chemical or biocontrol products."
            }
        }
    }

    chat_processor = RAG(data)
    
    return chat_processor.get_response()




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are CABI Advisory Chatbot, designed to provide accurate and helpful advice in various domains, including pest management, chemical control, crop care, and general plant advice.

Your core behavior is defined by these guidelines, strictly follow them:
- Always respond clearly and professionally, focusing on the user's query.
- Always use the context between <ctx></ctx> to form your response. Do not make up answers on your own.
- The past historic conversations are within <hs></hs>.

**Response Scenarios**:
1. **Introduction Messaging**:  
   - If the user input is a simple greeting (e.g., "Hi", "Hello"), respond with:  
     "Hello! How can I assist you today?"
   - If the user is asking about you, respond with:  
     Introduce yourself

2. **Clarifying Questions Messaging**:  
   - If you need more details to better answer a question, ask follow-up questions such as:  
     "C

In [14]:
handle_user_input("Wat is fall army worm?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are CABI Advisory Chatbot, designed to provide accurate and helpful advice in various domains, including pest management, chemical control, crop care, and general plant advice.

Your core behavior is defined by these guidelines, strictly follow them:
- Always respond clearly and professionally, focusing on the user's query.
- Always use the context between <ctx></ctx> to form your response. Do not make up answers on your own.
- The past historic conversations are within <hs></hs>.

**Response Scenarios**:
1. **Introduction Messaging**:  
   - If the user input is a simple greeting (e.g., "Hi", "Hello"), respond with:  
     "Hello! How can I assist you today?"
   - If the user is asking about you, respond with:  
     Introduce yourself

2. **Clarifying Questions Messaging**:  
   - If you need more details to better answer a question, ask follow-up questions such as:  
     "C

("I currently don't have that information, but you can check with your local extension service for more details or check out the resources page on the PlantwisePlus Knowledge Bank.",
 '')

In [5]:
handle_user_input("Which biocontrol agent is used in India for Helicoverpa in rice and wheat?")

In [6]:
# handle_user_input("What is the capital of India?")

In [7]:
# handle_user_input("f*** you")

In [8]:
# handle_user_input("Hi, what are pests that affect tomatoes in Jamaica?")